## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de termes MeSH présents dans les données / Termes de la taxonomie présents dans les données

### **Lire le corpus** 

In [81]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
import pandas as pd
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [82]:
lng = 'fr'
acteur = 'doctissimo' # Paramètre à indiquer avant de rouler le script
file = acteur +'.csv'

In [84]:
file

'doctissimo.csv'

In [85]:
data = pd.read_csv('../'+file)
data

,original,raw_sentences
0,https://forum.doctissimo.fr/experts/sante,"['', '\nConcours\n', ""\n\n\n\n\n\n\n\n\n\n\n\n..."
1,https://forum.doctissimo.fr/forme-beaute/Beaut...,"['', ""La levure de bière n'accélère pas la pou..."
2,https://forum.doctissimo.fr/forme-beaute/Beaut...,"['', ""Mais quel pub masque !! Je te dis je sui..."
3,https://forum.doctissimo.fr/forme-beaute/Beaut...,"['', '\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\tYour bro..."
4,https://forum.doctissimo.fr/forme-beaute/Beaut...,"['', ""Une des principales fonction du gel d'al..."
...,...,...
16211,https://forum.doctissimo.fr/sante/ventre-estom...,"['', 'bonjours je ne sait pas comment abordé s..."
16212,https://forum.doctissimo.fr/sante/ventre-estom...,"['', '[h3]27 Causes of Stomachaches—And How to..."
16213,https://forum.doctissimo.fr/sante/ventre-estom...,"['', ""Tu as raison, c'est tôt... je devrais pr..."
16214,https://forum.doctissimo.fr/sante/ventre-estom...,"['', ""Bonjour,Pour info on m'a enlevé la vésic..."


In [86]:
print("On a un corpus de {} documents.".format(len(data)))

On a un corpus de 16216 documents.


In [ ]:
data = data[:len(data)/2]
print("On a un corpus de {} documents.".format(len(data)))

### **Nettoyage**

In [88]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['raw_sentences'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Filtrage**
On va filtrer tout de suite filtrer certaines expressions relatives à l'architecture d'information / navigation Web

In [90]:
def filter_mwesw(corpus):
    file_mwesw = '../02-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [91]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [92]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus ; en théorie, le Tree Tagger est capable d'étiquetter
# environ 15 000 tokens / seconde. 
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))
    return tokens

tok(corpus)[:10]

Avec le RegExpTokenizer, notre corpus contient 7496801 tokens.
Le POS tagging devrait prendre environ 8 minutes.


['STOP',
 'STOP',
 'ah',
 'oui',
 'pour',
 'les',
 'logement',
 'sociaux',
 "c'",
 'est']

In [93]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output

In [94]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [95]:
tagged

[['STOP', 'NOM'],
 ["'", 'PUN'],
 ["'", 'PUN'],
 ['STOP', 'NOM'],
 ['"', 'PUN:cit'],
 ['ah', 'INT'],
 ['oui', 'INT'],
 ['pour', 'PRP'],
 ['les', 'DET:ART'],
 ['logement', 'NOM'],
 ['sociaux', 'ADJ'],
 ["c'", 'PRO:DEM'],
 ['est', 'VER:pres'],
 ['assez', 'ADV'],
 ['long', 'ADV'],
 ['dns-remplacé', 'VER:pper'],
 ('STOP', 'NAM'),
 ['on', 'PRO:PER'],
 ['va', 'VER:pres'],
 ['commencer', 'VER:infi'],
 ['des', 'PRP:det'],
 ['travaux', 'NOM'],
 ['dans', 'PRP'],
 ['notre', 'DET:POS'],
 ['maison', 'NOM'],
 ['STOP', 'NOM'],
 ['on', 'PRO:PER'],
 ['aménage', 'VER:pres'],
 ['des', 'PRP:det'],
 ['chambres', 'NOM'],
 ['dans', 'PRP'],
 ['le', 'DET:ART'],
 ['grenier', 'NOM'],
 ['pour', 'PRP'],
 ['nos', 'DET:POS'],
 ['grands', 'ADJ'],
 ['.', 'SENT'],
 ['et', 'KON'],
 ['bébé', 'NOM'],
 ['récupérera', 'VER:futu'],
 ['une', 'DET:ART'],
 ['chambre', 'NOM'],
 ['du', 'PRP:det'],
 ['STOP', 'NOM'],
 ['.', 'SENT'],
 ['je', 'PRO:PER'],
 ['pense', 'VER:pres'],
 ['que', 'KON'],
 ['ce', 'PRO:DEM'],
 ['ne', 'ADV'],
 ['

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [96]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=2, max_len=8))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [97]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 54029234 ngrammes.


In [98]:
ngrammes

[(['STOP', 'NOM'], ["'", 'PUN']),
 (['STOP', 'NOM'], ["'", 'PUN'], ["'", 'PUN']),
 (['STOP', 'NOM'], ["'", 'PUN'], ["'", 'PUN'], ['STOP', 'NOM']),
 (['STOP', 'NOM'],
  ["'", 'PUN'],
  ["'", 'PUN'],
  ['STOP', 'NOM'],
  ['"', 'PUN:cit']),
 (['STOP', 'NOM'],
  ["'", 'PUN'],
  ["'", 'PUN'],
  ['STOP', 'NOM'],
  ['"', 'PUN:cit'],
  ['ah', 'INT']),
 (['STOP', 'NOM'],
  ["'", 'PUN'],
  ["'", 'PUN'],
  ['STOP', 'NOM'],
  ['"', 'PUN:cit'],
  ['ah', 'INT'],
  ['oui', 'INT']),
 (['STOP', 'NOM'],
  ["'", 'PUN'],
  ["'", 'PUN'],
  ['STOP', 'NOM'],
  ['"', 'PUN:cit'],
  ['ah', 'INT'],
  ['oui', 'INT'],
  ['pour', 'PRP']),
 (["'", 'PUN'], ["'", 'PUN']),
 (["'", 'PUN'], ["'", 'PUN'], ['STOP', 'NOM']),
 (["'", 'PUN'], ["'", 'PUN'], ['STOP', 'NOM'], ['"', 'PUN:cit']),
 (["'", 'PUN'],
  ["'", 'PUN'],
  ['STOP', 'NOM'],
  ['"', 'PUN:cit'],
  ['ah', 'INT']),
 (["'", 'PUN'],
  ["'", 'PUN'],
  ['STOP', 'NOM'],
  ['"', 'PUN:cit'],
  ['ah', 'INT'],
  ['oui', 'INT']),
 (["'", 'PUN'],
  ["'", 'PUN'],
  ['STOP',

### **Extraction des patrons syntaxiques**

In [99]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [100]:
phrases = extract_patterns(ngrammes)

In [101]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=8))

In [102]:
frequencies

FreqDist({('STOP',): 748938, ('de',): 205394, ('je',): 142012, ('et',): 136742, ('.',): 126438, ('STOP', 'STOP'): 125571, ('"',): 124191, ('le',): 111338, ("'",): 106264, ('la',): 102137, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [103]:
# Importer l'antidictionnaire pour filtrer les données

# Stopwords fréquents en français (non lemmatisés)
file_path = "../02-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../02-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

In [104]:
def filtrer_stopwords(x): 
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [105]:
phrases = filtrer_stopwords(phrases)

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [106]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

phrases = filter_len(phrases)

On retire les n-grammes qui apparaissent moins de N fois dans le corpus

In [107]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 5]

#phrases = filter_freq(phrases)

In [108]:
phrases

[[('oui', 'pour', 'les', 'logement'), ['INT', 'PRP', 'DET:ART', 'NOM']],
 [('oui', 'pour', 'les', 'logement', 'sociaux'),
  ['INT', 'PRP', 'DET:ART', 'NOM', 'ADJ']],
 [('logement', 'sociaux'), ['NOM', 'ADJ']],
 [('logement', 'sociaux', "c'", 'est', 'assez', 'long'),
  ['NOM', 'ADJ', 'PRO:DEM', 'VER:pres', 'ADV', 'ADV']],
 [('sociaux', "c'", 'est', 'assez', 'long'),
  ['ADJ', 'PRO:DEM', 'VER:pres', 'ADV', 'ADV']],
 [('commencer', 'des', 'travaux'), ['VER:infi', 'PRP:det', 'NOM']],
 [('commencer', 'des', 'travaux', 'dans', 'notre', 'maison'),
  ['VER:infi', 'PRP:det', 'NOM', 'PRP', 'DET:POS', 'NOM']],
 [('travaux', 'dans', 'notre', 'maison'), ['NOM', 'PRP', 'DET:POS', 'NOM']],
 [('aménage', 'des', 'chambres'), ['VER:pres', 'PRP:det', 'NOM']],
 [('chambres', 'dans', 'le', 'grenier', 'pour', 'nos', 'grands'),
  ['NOM', 'PRP', 'DET:ART', 'NOM', 'PRP', 'DET:POS', 'ADJ']],
 [('bébé', 'récupérera'), ['NOM', 'VER:futu']],
 [('bébé', 'récupérera', 'une', 'chambre'),
  ['NOM', 'VER:futu', 'DET:AR

In [109]:
phrases = [[term[0], " ".join(term[1])] for term in phrases]

In [110]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). *Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches*. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [111]:
file_patterns = '../02-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = pd.read_csv(f)
    patterns = patterns['Structure'].tolist() # Pour prendre les structures syntaxiques attestées dans les MeSH

In [112]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]

In [113]:
terms = filter_patterns(phrases)

In [114]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 72 % des termes
On avait 3128573 ngrammes, on en a maintenant 865578.


In [115]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = pd.DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [116]:
terms_patterns = [t for t in terms_patterns if t['Fréquence'] > 2]
for x in terms_patterns :
    x['Expression'] = " ".join(x['Expression']).replace("' ", "'").replace("( ", "(").replace(" )", ")").replace(' -', '-') 

### **Extraction de termes MeSH**

In [117]:
df = pd.DataFrame(terms_patterns).drop_duplicates

In [118]:
df['isMeSHTerm']= False # On set à False puis on va changer pour True si on trouve le terme
df['MeSHID'] = None
df['MesH_prefLabel_fr'] = None
df['MesH_prefLabel_en'] = None

In [119]:
from nltk.tokenize import MWETokenizer
file_path = '../02-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [120]:
df

,Expression,Structure syntaxique,Fréquence,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,temps qu',NOM PRO:REL,83,False,None,None,None
1,liste de naissance,NOM PRP NOM,26,False,None,None,None
2,liste de naissance,NOM PRP NOM,26,False,None,None,None
3,également acheté,ADV VER:pper,7,False,None,None,None
4,attend tjrs,VER:pres NOM,3,False,None,None,None
...,...,...,...,...,...,...,...
202043,vinaigre de cidre,NOM PRP NOM,34,False,None,None,None
202044,prendre un traitement,VER:infi DET:ART NOM,28,False,None,None,None
202045,bonne érection,ADJ NOM,3,False,None,None,None
202046,quasi impossible,ADV ADJ,15,False,None,None,None


In [121]:
list_terms = df['Expression'].tolist()
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [122]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Expression'] == t, 'isMeSHTerm'] = True

df = df.drop_duplicates(subset = "Expression")
df

,Expression,Structure syntaxique,Fréquence,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
0,temps qu',NOM PRO:REL,83,False,None,None,None
1,liste de naissance,NOM PRP NOM,26,False,None,None,None
3,également acheté,ADV VER:pper,7,False,None,None,None
4,attend tjrs,VER:pres NOM,3,False,None,None,None
5,permis de construire,NOM PRP VER:infi,4,False,None,None,None
...,...,...,...,...,...,...,...
201464,alcaline ionisée,ADJ VER:pper,3,False,None,None,None
201560,droits de l'homme,NOM PRP DET:ART NOM,4,False,None,None,None
201568,nations unies,NOM ADJ,4,True,None,None,None
201741,promenade privative,NOM ADJ,3,False,None,None,None


In [123]:
# Termes MeSH présents dans notre corpus 
df[df['isMeSHTerm'] == True]

,Expression,Structure syntaxique,Fréquence,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en
7,fausse couche,ADJ NOM,474,True,None,None,None
12,grand mère,ADJ NOM,82,True,None,None,None
22,point de vue,NOM PRP NOM,187,True,None,None,None
57,ulcère cornéen,NOM ADJ,4,True,None,None,None
61,larmes artificielles,NOM ADJ,18,True,None,None,None
...,...,...,...,...,...,...,...
189566,candida glabrata,NOM VER:simp,6,True,None,None,None
190285,syndrome des ongles jaunes,NOM PRP:det NOM ADJ,6,True,None,None,None
192198,marqueur tumoral,NOM ADJ,3,True,None,None,None
193558,carence en acide folique,NOM PRP NOM ADJ,3,True,None,None,None


### **Extraction de termes existant dans la taxonomie**

In [124]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme

/var/folders/99/r0skp4wd6dx1rrw6kjr2nmz00000gq/T/ipykernel_16078/2422679389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme


In [125]:
file_path = '../02-filtrage/default_taxo_labels.csv'

default = pd.read_csv(file_path)
taxo_terms = list(dict.fromkeys([str(t).strip().lower() for t in default['label'].tolist()]))

taxo_terms

['neurochirurgie stereotaxique et fonctionnelle',
 'stereotactic and functional neurosurgery',
 'neurosurgery',
 'neurochirurgie',
 'deep brain stimulation',
 'stimulation cérébrale profonde',
 'poly polymerase inhibitors',
 'other antineoplastic agents',
 'lifestylemodifications',
 'familymedicineinterventions',
 'healthpromotionprevention',
 'psychologicalbehavioralti',
 'alliedhealthfacility',
 'établissement de soins ambulatoires',
 'ambulatorycaremedicalfacility',
 'ambulatory care facility',
 'medicalfacilitytype',
 'languagecognitionfacility',
 'prostesisorthesisfacility',
 'earauditionfacility',
 'nutritiondietfacility',
 'eyevisionfacility',
 'mobilitymusculoskeletalfacility',
 'garderie',
 'nursery',
 'parietal lobe',
 'lobe pariétal',
 'cortex cérébral',
 'cerebral cortex',
 'cognitive dissonance',
 'dissonance cognitive',
 'cognition',
 'manometry',
 'manométrie',
 'digestivefonctiontests',
 'manométrie anorectale',
 'anorectal manometry',
 'esophageal manometry',
 'manomét

In [126]:
for t in df['Expression'].tolist():
    if t in taxo_terms:
        df.loc[df['Expression'] == t, 'isTaxoTerm'] = True
df[df['isTaxoTerm'] == True]

,Expression,Structure syntaxique,Fréquence,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm
7,fausse couche,ADJ NOM,474,True,None,None,None,True
18,couches lavables,NOM ADJ,69,False,None,None,None,True
61,larmes artificielles,NOM ADJ,18,True,None,None,None,True
100,yeux secs,NOM ADJ,12,False,None,None,None,True
105,test de grossesse,NOM PRP NOM,170,True,None,None,None,True
...,...,...,...,...,...,...,...,...
163139,capacités d'adaptation,NOM PRP NOM,3,False,None,None,None,True
163146,prévention des infections,NOM PRP:det NOM,3,True,None,None,None,True
166704,pompe à insuline,NOM PRP NOM,3,False,None,None,None,True
190285,syndrome des ongles jaunes,NOM PRP:det NOM ADJ,6,True,None,None,None,True


### **Mapping to MeSH IDs**

Lire le fichier XML contenant le MeSH Bilingue

Le MeSH 2019 comprend 29 351 descripteurs  
https://mesh.inserm.fr/FrenchMesh/presentation.htm

In [127]:
import xml.etree.ElementTree as ET
import re
import pandas as pd
base_path = '../02-filtrage/MeSH/'

tree = ET.parse(base_path + 'fredesc2019.xml')
root = tree.getroot()

def flatten(l):
    return [item for sublist in l for item in sublist]

data_mesh = [{'mesh_id' : x.find('DescriptorUI').text.strip('\n'), \
         'label_fr' : x.find('DescriptorName').find('String').text.split('[')[0], \
         'label_en' : x.find('DescriptorName').find('String').text.split('[')[1].strip(']'), \
         'synonymes (en/fr)' : flatten([[term.find('String').text for term in concept.find('TermList').findall('Term')] for concept in x.find('ConceptList').findall('Concept')]) \
         } for x in root.findall('DescriptorRecord')]

In [128]:
for t in df[df['isMeSHTerm'] == True]['Expression'].tolist():
    for d in data_mesh:
        if t in [str(x).lower() for x in d['synonymes (en/fr)']]:
            df.loc[df['Expression'] == t, 'MeSHID'] = d['mesh_id']
            df.loc[df['Expression'] == t, 'MesH_prefLabel_fr'] = d['label_fr']
            df.loc[df['Expression'] == t, 'MesH_prefLabel_en'] = d['label_en']

df

,Expression,Structure syntaxique,Fréquence,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm
0,temps qu',NOM PRO:REL,83,False,None,None,None,False
1,liste de naissance,NOM PRP NOM,26,False,None,None,None,False
3,également acheté,ADV VER:pper,7,False,None,None,None,False
4,attend tjrs,VER:pres NOM,3,False,None,None,None,False
5,permis de construire,NOM PRP VER:infi,4,False,None,None,None,False
...,...,...,...,...,...,...,...,...
201464,alcaline ionisée,ADJ VER:pper,3,False,None,None,None,False
201560,droits de l'homme,NOM PRP DET:ART NOM,4,False,None,None,None,True
201568,nations unies,NOM ADJ,4,True,D014480,Nations Unies,United Nations,False
201741,promenade privative,NOM ADJ,3,False,None,None,None,False


In [129]:
df.insert(0, 'Corpus', acteur)
df

,Corpus,Expression,Structure syntaxique,Fréquence,isMeSHTerm,MeSHID,MesH_prefLabel_fr,MesH_prefLabel_en,isTaxoTerm
0,doctissimo,temps qu',NOM PRO:REL,83,False,None,None,None,False
1,doctissimo,liste de naissance,NOM PRP NOM,26,False,None,None,None,False
3,doctissimo,également acheté,ADV VER:pper,7,False,None,None,None,False
4,doctissimo,attend tjrs,VER:pres NOM,3,False,None,None,None,False
5,doctissimo,permis de construire,NOM PRP VER:infi,4,False,None,None,None,False
...,...,...,...,...,...,...,...,...,...
201464,doctissimo,alcaline ionisée,ADJ VER:pper,3,False,None,None,None,False
201560,doctissimo,droits de l'homme,NOM PRP DET:ART NOM,4,False,None,None,None,True
201568,doctissimo,nations unies,NOM ADJ,4,True,D014480,Nations Unies,United Nations,False
201741,doctissimo,promenade privative,NOM ADJ,3,False,None,None,None,False


In [130]:
df.to_csv('../' + acteur + '_candidate-terms.csv')